In [ ]:
!fusermount -u /content/drive
!rm -rf /content/drive

fusermount: failed to unmount /content/drive: No such file or directory


# Blocks Approach

In [ ]:
import pandas as pd
from datetime import datetime, timedelta
import math
import os


try:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)
    BASE_PATH = "/content/drive/My Drive/ASCP PW/2. BLOCKS"
except ImportError:
    BASE_PATH = "./" # Adjust if your script is elsewhere relative to data/output

INPUT_FOLDER = os.path.join(BASE_PATH, "TRIPS_PER_DAY_CSV") # Assuming CSVs are directly in BASE_PATH
OUTPUT_FOLDER = os.path.join(BASE_PATH, "Blocks Analysis")

ORDERS_FILE = os.path.join(INPUT_FOLDER, "1603.csv")
DISTANCES_FILE = os.path.join(INPUT_FOLDER, "Driving_Distances.csv")


BASE_LOCATION = 'BIA'

# Average truck speeds (km/h)
SPEED_UP_TO_25KM = 30
SPEED_25_TO_50KM = 37.5  # Using average of 35-40 km/h
SPEED_50_TO_100KM = 50
SPEED_ABOVE_100KM = 60

# Operations times (minutes)
OPERATIONS_TIME_FV = 75
OPERATIONS_TIME_LAS = 60
OPERATIONS_TIME_DROGE = 45
OPERATIONS_TIME_EXTRA = 30

# Waiting time per point (minutes)
WAITING_TIME_PER_POINT = 15

# Special code time (minutes)
SPECIAL_CODE_TIME_VALUE = 45
SPECIAL_CODES_LIST = ['BV', 'RPV', 'V', 'VUOTI']

# Start time flexibility (minutes)
FLEXIBILITY_LAS = 0
FLEXIBILITY_FV = 15
FLEXIBILITY_DROGE = 15
FLEXIBILITY_EXTRA = 30

# Constraints (minutes)
MAX_DAILY_DRIVING_STD = 9 * 60
MAX_DAILY_DRIVING_EXT = 10 * 60
MAX_CONTINUOUS_DRIVING = 4.5 * 60
MAX_DAILY_WORKING_STD = 13 * 60
MAX_DAILY_WORKING_EXT = 15 * 60
MIN_DAILY_WORKING = 8 * 60


# --- Helper Functions ---

def get_distance_lookup(distances_file_path):
    """Loads distances into a lookup dictionary."""
    try:
        distances_df = pd.read_csv(distances_file_path)
        lookup = {}
        for _, row in distances_df.iterrows():
            p1_clean = str(row['ORIGIN']).strip()
            p2_clean = str(row['DESTINATION']).strip()
            pair1 = tuple(sorted((p1_clean, p2_clean)))
            lookup[pair1] = float(row['DISTANCE'])
        return lookup
    except FileNotFoundError:
        print(f"Error: Distances file not found at {distances_file_path}")
        return None
    except Exception as e:
        print(f"Error loading distances: {e}")
        return None

_distances_lookup = None # Global cache for distances

def get_distance(p1, p2):
    """Gets distance between two points using the lookup."""
    global _distances_lookup
    if _distances_lookup is None:
        print("Error: Distances lookup not initialized.")
        return float('inf')

    p1_clean = str(p1).strip()
    p2_clean = str(p2).strip()

    if p1_clean == p2_clean:
        return 0.0

    pair = tuple(sorted((p1_clean, p2_clean)))
    return _distances_lookup.get(pair, float('inf'))

def calculate_segment_driving_time(distance_km):
    """Calculates driving time in minutes for a segment based on distance."""
    if distance_km == 0:
        return 0
    if distance_km == float('inf'):
        return float('inf')

    speed_kmh = 0
    if distance_km <= 25:
        speed_kmh = SPEED_UP_TO_25KM
    elif distance_km <= 50:
        speed_kmh = SPEED_25_TO_50KM
    elif distance_km <= 100:
        speed_kmh = SPEED_50_TO_100KM
    else:
        speed_kmh = SPEED_ABOVE_100KM

    if speed_kmh == 0: return float('inf')
    return math.ceil((distance_km / speed_kmh) * 60)

def get_order_operations_time_val(order_type, description):
    """Gets operations time for an order."""
    if order_type == 'Standard':
        if description == 'FV': return OPERATIONS_TIME_FV
        elif description == 'LAS': return OPERATIONS_TIME_LAS
        elif description == 'DROGE': return OPERATIONS_TIME_DROGE
    elif order_type == 'Extra':
        return OPERATIONS_TIME_EXTRA
    return 0

def get_order_total_waiting_time_val(points_list):
    """Gets total waiting time for an order."""
    num_non_bia_points = 0
    for point in points_list:
        if point and str(point).strip() != BASE_LOCATION:
            num_non_bia_points += 1
    return num_non_bia_points * WAITING_TIME_PER_POINT

def get_start_flexibility(order_type, description):
    """Gets start time flexibility in minutes."""
    if order_type == 'Standard':
        if description == 'LAS': return FLEXIBILITY_LAS
        if description == 'FV': return FLEXIBILITY_FV
        if description == 'DROGE': return FLEXIBILITY_DROGE
    elif order_type == 'Extra':
        return FLEXIBILITY_EXTRA
    return 0

def parse_datetime(time_str):
    """Parses date string into datetime object."""
    try:
        return datetime.strptime(str(time_str).strip(), '%d/%m/%Y %H:%M')
    except ValueError:
        try:
            return datetime.strptime(str(time_str).strip(), '%d/%m/%Y')
        except ValueError:
            # print(f"Warning: Could not parse datetime string: {time_str}") # Reduced verbosity
            return None


# --- Main Algorithm ---
def optimize_deliveries():
    global _distances_lookup
    _distances_lookup = get_distance_lookup(DISTANCES_FILE)
    if _distances_lookup is None:
        print("Exiting due to distance lookup initialization failure.")
        return

    try:
        # Attempt to read with a common encoding, and fall back if needed.
        try:
            orders_df_initial = pd.read_csv(ORDERS_FILE, encoding='utf-8')
        except UnicodeDecodeError:
            print("UTF-8 decoding failed, trying with 'latin1' encoding for orders file.")
            orders_df_initial = pd.read_csv(ORDERS_FILE, encoding='latin1')
    except FileNotFoundError:
        print(f"Error: Orders file not found at {ORDERS_FILE}")
        return
    except Exception as e:
        print(f"Error loading orders CSV: {e}")
        return

    # Pre-process orders
    processed_orders = []
    skipped_orders_due_to_time_parse = 0
    for idx, order_row in orders_df_initial.iterrows():
        order = order_row.to_dict()
        # Store the original DataFrame index, which might be more stable than 'id' if 'id' is not unique or missing
        order['original_order_id'] = order.get('ID', idx) # Prefer 'ID' column if exists, else use df index
        order['original_df_index'] = idx


        order['TYPE'] = str(order.get('TYPE', '')).strip()
        order['DESCRIPTION'] = str(order.get('DESCRIPTION', '')).strip()
        order['NOTES'] = str(order.get('NOTES', '')).strip()
        order['START_original_str'] = str(order.get('START', '')) # Keep original start string for unassigned

        order['start_datetime_orig'] = parse_datetime(order.get('START'))
        if order['start_datetime_orig'] is None:
            # print(f"Skipping order original_id={order['original_order_id']} due to invalid START time: {order.get('START')}")
            skipped_orders_due_to_time_parse +=1
            continue

        order['flexibility_minutes'] = get_start_flexibility(order['TYPE'], order['DESCRIPTION'])
        order['earliest_start_dt'] = order['start_datetime_orig']
        order['latest_start_dt'] = order['start_datetime_orig'] + timedelta(minutes=order['flexibility_minutes'])

        points = []
        for i in range(1, 7):
            pt_val = str(order.get(f'POINT_{i}', '')).strip()
            if pt_val and pd.notna(pt_val) and pt_val.lower() != 'nan' and pt_val.lower() != '':
                points.append(pt_val)
        order['points_list_orig'] = list(points)

        order_special_code_time = 0
        current_points_for_processing = list(points) # Work with a copy
        if any(code in order['NOTES'] for code in SPECIAL_CODES_LIST):
            order_special_code_time = SPECIAL_CODE_TIME_VALUE
            if not current_points_for_processing or current_points_for_processing[-1] != BASE_LOCATION:
                if len(current_points_for_processing) < 6:
                    current_points_for_processing.append(BASE_LOCATION)
                else:
                    current_points_for_processing[-1] = BASE_LOCATION
        order['points_list_final'] = list(current_points_for_processing) # Use the potentially modified list
        order['inherent_special_code_time'] = order_special_code_time

        order_trip_driving_time = 0
        if not order['points_list_final']:
             # print(f"Warning: Order original_id={order['original_order_id']} has no points in points_list_final.")
             pass # Driving time remains 0
        elif len(order['points_list_final']) >= 2:
            for i in range(len(order['points_list_final']) - 1):
                p1 = order['points_list_final'][i]
                p2 = order['points_list_final'][i+1]
                dist = get_distance(p1, p2)
                if dist == float('inf'):
                    # print(f"Warning: Missing distance for order original_id={order['original_order_id']} between {p1} and {p2}.")
                    order_trip_driving_time = float('inf')
                    break
                order_trip_driving_time += calculate_segment_driving_time(dist)

        if order_trip_driving_time == float('inf'):
            # print(f"Skipping order original_id={order['original_order_id']} due to missing distance in its path.")
            # This order will be considered unassigned later
            order['reason_unassigned'] = "Missing distance in its path"
            processed_orders.append(order) # Add to processed so it can be listed as unassigned
            continue


        order['inherent_trip_driving_time'] = order_trip_driving_time
        order['inherent_operations_time'] = get_order_operations_time_val(order['TYPE'], order['DESCRIPTION'])
        order['inherent_total_waiting_time'] = get_order_total_waiting_time_val(order['points_list_final'])

        order['fixed_duration_components'] = (order['inherent_trip_driving_time'] +
                                              order['inherent_operations_time'] +
                                              order['inherent_total_waiting_time'] +
                                              order['inherent_special_code_time'])
        order['reason_unassigned'] = '' # Initialize, will be set if assignment fails
        processed_orders.append(order)

    if skipped_orders_due_to_time_parse > 0:
        print(f"Note: Skipped {skipped_orders_due_to_time_parse} orders during pre-processing due to invalid START times.")

    processed_orders.sort(key=lambda o: (o['earliest_start_dt'], o['original_order_id']))

    drivers_plan = []
    assigned_order_ids = set() # Store original_order_id
    driver_id_counter = 0 # This will be used for Block ID

    # Main Assignment Loop
    while True:
        made_assignment_in_iteration = False
        best_assignment_for_existing_driver = None
        best_assignment_score = float('inf')

        eligible_orders_for_assignment = [o for o in processed_orders if o['original_order_id'] not in assigned_order_ids and o.get('inherent_trip_driving_time', 0) != float('inf')]


        if not eligible_orders_for_assignment:
            # print("No more eligible orders to assign.")
            break

        # Try to assign to an existing driver
        for driver_idx, driver_block in enumerate(drivers_plan):
            if driver_block.get('finalized', False): continue

            last_order_in_block = driver_block['orders'][-1]
            prev_order_actual_end_time = last_order_in_block['actual_end_time']

            if not last_order_in_block['points_list_final']:
                continue
            prev_order_last_point = last_order_in_block['points_list_final'][-1]

            for order_to_assign in eligible_orders_for_assignment:
                if not order_to_assign['points_list_final']:
                    continue
                next_order_first_point = order_to_assign['points_list_final'][0]

                deadhead_dist = get_distance(prev_order_last_point, next_order_first_point)
                if deadhead_dist == float('inf'):
                    order_to_assign['reason_unassigned'] = f"Missing deadhead from {prev_order_last_point} to {next_order_first_point}"
                    continue
                deadhead_time_to_next = calculate_segment_driving_time(deadhead_dist)

                earliest_arrival_at_next_p1 = prev_order_actual_end_time + timedelta(minutes=deadhead_time_to_next)
                actual_start_for_next_order = max(earliest_arrival_at_next_p1, order_to_assign['earliest_start_dt'])

                if actual_start_for_next_order > order_to_assign['latest_start_dt']:
                    order_to_assign['reason_unassigned'] = f"Cannot meet delivery window (latest: {order_to_assign['latest_start_dt']}, earliest possible: {actual_start_for_next_order})"
                    continue

                idle_time_val = max(0, (actual_start_for_next_order - earliest_arrival_at_next_p1).total_seconds() / 60)

                temp_return_to_bia_time = 0
                if order_to_assign['points_list_final'][-1] != BASE_LOCATION:
                    dist_to_bia = get_distance(order_to_assign['points_list_final'][-1], BASE_LOCATION)
                    if dist_to_bia != float('inf'):
                        temp_return_to_bia_time = calculate_segment_driving_time(dist_to_bia)
                    else:
                        temp_return_to_bia_time = float('inf') # Makes this assignment invalid for checks

                proj_trip_driving = driver_block['sum_trip_driving_time'] + order_to_assign['inherent_trip_driving_time']
                proj_deadhead_driving = driver_block['sum_deadhead_driving_time'] + deadhead_time_to_next
                proj_total_driving = proj_trip_driving + proj_deadhead_driving + temp_return_to_bia_time

                proj_ops = driver_block['sum_operations_time'] + order_to_assign['inherent_operations_time']
                proj_wait = driver_block['sum_waiting_time'] + order_to_assign['inherent_total_waiting_time']
                proj_special = driver_block['sum_special_code_time'] + order_to_assign['inherent_special_code_time']
                proj_pure_idle = driver_block['sum_pure_idle_time'] + idle_time_val
                proj_total_working = proj_total_driving + proj_ops + proj_wait + proj_special + proj_pure_idle

                # Continuous Driving Check
                current_order_inherent_driving = order_to_assign['inherent_trip_driving_time']
                if idle_time_val > 0 or not driver_block['last_activity_was_driving']:
                    accumulated_continuous_driving_for_order = deadhead_time_to_next + current_order_inherent_driving
                else:
                    accumulated_continuous_driving_for_order = driver_block['continuous_driving_accumulator'] + deadhead_time_to_next + current_order_inherent_driving

                if accumulated_continuous_driving_for_order > MAX_CONTINUOUS_DRIVING:
                    order_to_assign['reason_unassigned'] = f"Exceeds max continuous driving ({accumulated_continuous_driving_for_order} > {MAX_CONTINUOUS_DRIVING})"
                    continue

                fits_std = (proj_total_driving <= MAX_DAILY_DRIVING_STD and proj_total_working <= MAX_DAILY_WORKING_STD)
                fits_ext = (proj_total_driving <= MAX_DAILY_DRIVING_EXT and proj_total_working <= MAX_DAILY_WORKING_EXT)

                can_assign = False
                used_extended = False
                if fits_std: can_assign = True
                elif fits_ext: can_assign = True; used_extended = True

                if can_assign:
                    score = idle_time_val + deadhead_time_to_next
                    if used_extended: score += 1000

                    if score < best_assignment_score:
                        best_assignment_score = score
                        best_assignment_for_existing_driver = {
                            'order': order_to_assign, 'driver_idx': driver_idx, 'score': score,
                            'actual_start': actual_start_for_next_order,
                            'idle_time': idle_time_val, 'deadhead_time': deadhead_time_to_next,
                            'continuous_driving_val_for_order': accumulated_continuous_driving_for_order,
                            'used_extended': used_extended
                        }

        if best_assignment_for_existing_driver:
            assign_details = best_assignment_for_existing_driver
            order = assign_details['order']
            driver_block = drivers_plan[assign_details['driver_idx']]

            order['actual_start_time'] = assign_details['actual_start']
            order['actual_end_time'] = order['actual_start_time'] + timedelta(minutes=order['fixed_duration_components'])

            driver_block['orders'].append(order)
            driver_block['sum_trip_driving_time'] += order['inherent_trip_driving_time']
            driver_block['sum_deadhead_driving_time'] += assign_details['deadhead_time']
            driver_block['sum_operations_time'] += order['inherent_operations_time']
            driver_block['sum_waiting_time'] += order['inherent_total_waiting_time']
            driver_block['sum_special_code_time'] += order['inherent_special_code_time']
            driver_block['sum_pure_idle_time'] += assign_details['idle_time']

            driver_block['continuous_driving_accumulator'] = assign_details['continuous_driving_val_for_order']
            driver_block['last_activity_was_driving'] = not (order['inherent_operations_time'] > 0 or order['inherent_total_waiting_time'] > 0 or order['inherent_special_code_time'] > 0)
            if not driver_block['last_activity_was_driving']:
                driver_block['continuous_driving_accumulator'] = 0

            if assign_details['used_extended']:
                driver_block['used_extended_hours'] = True

            assigned_order_ids.add(order['original_order_id'])
            made_assignment_in_iteration = True

        else: # No existing driver could take any of the remaining orders, try to start a new one
            order_for_new_driver = None
            best_new_driver_order_start_time = datetime.max
            temp_new_driver_assignment_details = {}

            for order_to_assign in eligible_orders_for_assignment: # Already filtered for not assigned and valid path
                if not order_to_assign['points_list_final']:
                    continue

                initial_deadhead_time = 0
                if order_to_assign['points_list_final'][0] != BASE_LOCATION:
                    initial_deadhead_dist = get_distance(BASE_LOCATION, order_to_assign['points_list_final'][0])
                    if initial_deadhead_dist == float('inf'):
                        order_to_assign['reason_unassigned'] = f"Missing initial deadhead from BIA to {order_to_assign['points_list_final'][0]}"
                        continue
                    initial_deadhead_time = calculate_segment_driving_time(initial_deadhead_dist)

                actual_start_for_order = order_to_assign['earliest_start_dt']
                # No check against latest_start_dt here as it's the first order, assumed to be met.

                temp_return_to_bia_time = 0
                if order_to_assign['points_list_final'][-1] != BASE_LOCATION:
                    dist_to_bia = get_distance(order_to_assign['points_list_final'][-1], BASE_LOCATION)
                    if dist_to_bia != float('inf'):
                        temp_return_to_bia_time = calculate_segment_driving_time(dist_to_bia)
                    else: temp_return_to_bia_time = float('inf')

                proj_trip_driving = order_to_assign['inherent_trip_driving_time']
                proj_deadhead_driving = initial_deadhead_time
                proj_total_driving = proj_trip_driving + proj_deadhead_driving + temp_return_to_bia_time

                proj_ops = order_to_assign['inherent_operations_time']
                proj_wait = order_to_assign['inherent_total_waiting_time']
                proj_special = order_to_assign['inherent_special_code_time']
                proj_pure_idle = 0
                proj_total_working = proj_total_driving + proj_ops + proj_wait + proj_special + proj_pure_idle

                continuous_driving_segment = initial_deadhead_time + order_to_assign['inherent_trip_driving_time']
                if continuous_driving_segment > MAX_CONTINUOUS_DRIVING:
                    order_to_assign['reason_unassigned'] = f"Exceeds max continuous driving for new driver ({continuous_driving_segment})"
                    continue

                fits_std = (proj_total_driving <= MAX_DAILY_DRIVING_STD and proj_total_working <= MAX_DAILY_WORKING_STD)
                fits_ext = (proj_total_driving <= MAX_DAILY_DRIVING_EXT and proj_total_working <= MAX_DAILY_WORKING_EXT)

                can_assign_new = False
                used_extended_new = False
                if fits_std: can_assign_new = True
                elif fits_ext: can_assign_new = True; used_extended_new = True

                if can_assign_new:
                    if order_to_assign['earliest_start_dt'] < best_new_driver_order_start_time:
                        best_new_driver_order_start_time = order_to_assign['earliest_start_dt']
                        order_for_new_driver = order_to_assign
                        temp_new_driver_assignment_details = {
                            'order': order_to_assign, 'actual_start': actual_start_for_order,
                            'initial_deadhead': initial_deadhead_time,
                            'continuous_driving_val': continuous_driving_segment,
                            'used_extended': used_extended_new
                        }

            if order_for_new_driver:
                assign_details = temp_new_driver_assignment_details
                order = assign_details['order']
                driver_id_counter += 1 # This is now the Block ID

                order['actual_start_time'] = assign_details['actual_start']
                order['actual_end_time'] = order['actual_start_time'] + timedelta(minutes=order['fixed_duration_components'])

                new_block = {
                    'block_id': driver_id_counter,
                    'orders': [order],
                    'sum_trip_driving_time': order['inherent_trip_driving_time'],
                    'sum_deadhead_driving_time': assign_details['initial_deadhead'],
                    'sum_operations_time': order['inherent_operations_time'],
                    'sum_waiting_time': order['inherent_total_waiting_time'],
                    'sum_special_code_time': order['inherent_special_code_time'],
                    'sum_pure_idle_time': 0,
                    'return_trip_driving_time': 0,
                    'continuous_driving_accumulator': assign_details['continuous_driving_val'],
                    'last_activity_was_driving': not (order['inherent_operations_time'] > 0 or order['inherent_total_waiting_time'] > 0 or order['inherent_special_code_time'] > 0),
                    'used_extended_hours': assign_details['used_extended']
                }
                if not new_block['last_activity_was_driving']: new_block['continuous_driving_accumulator'] = 0

                drivers_plan.append(new_block)
                assigned_order_ids.add(order['original_order_id'])
                made_assignment_in_iteration = True

        if not made_assignment_in_iteration:
            # print("No assignment could be made in this iteration for any remaining orders.")
            break # Exit while loop if no progress

    # --- Prepare Output Data ---
    optimized_plan_rows = []
    for driver_block in drivers_plan: # driver_block is actually just 'block' now
        if not driver_block['orders']: continue

        last_order_of_block = driver_block['orders'][-1]
        final_location = BASE_LOCATION
        if last_order_of_block['points_list_final']:
            final_location = last_order_of_block['points_list_final'][-1]

        driver_return_trip_time = 0
        if final_location != BASE_LOCATION:
            dist = get_distance(final_location, BASE_LOCATION)
            if dist != float('inf'):
                driver_return_trip_time = calculate_segment_driving_time(dist)
                driver_block['return_trip_driving_time'] = driver_return_trip_time
            else:
                driver_block['return_trip_driving_time'] = 0
                # print(f"Warning: Could not calculate return to BIA for block {driver_block['block_id']} from {final_location}")

        driver_block['total_driving_time_final'] = (driver_block['sum_trip_driving_time'] +
                                                    driver_block['sum_deadhead_driving_time'] +
                                                    driver_block['return_trip_driving_time'])

        driver_block['total_working_time_final'] = (driver_block['total_driving_time_final'] +
                                                    driver_block['sum_operations_time'] +
                                                    driver_block['sum_waiting_time'] +
                                                    driver_block['sum_special_code_time'] +
                                                    driver_block['sum_pure_idle_time'])

        for trip_idx, trip_order in enumerate(driver_block['orders']):
            prev_loc_for_deadhead = BASE_LOCATION
            if trip_idx > 0:
                prev_trip_in_block = driver_block['orders'][trip_idx-1]
                if prev_trip_in_block['points_list_final']:
                     prev_loc_for_deadhead = prev_trip_in_block['points_list_final'][-1]

            current_trip_first_point = trip_order['points_list_final'][0] if trip_order['points_list_final'] else ""
            deadhead_to_this_trip = 0
            if current_trip_first_point and current_trip_first_point != prev_loc_for_deadhead :
                 dist_deadhead = get_distance(prev_loc_for_deadhead, current_trip_first_point)
                 if dist_deadhead != float('inf'):
                     deadhead_to_this_trip = calculate_segment_driving_time(dist_deadhead)

            row = {
                # 'Driver ID': driver_block['driver_id'], # Removed as per request
                'Block ID': driver_block['block_id'],
                'Trip Order in Block': trip_idx + 1,
                'Trip ID (Order ID)': trip_order['original_order_id'],
                'Type': trip_order['TYPE'],
                'Description': trip_order['DESCRIPTION'],
                'Truck Type': trip_order.get('TRUCK_TYPE', 'TR'),
                'Point 1': trip_order['points_list_final'][0] if len(trip_order['points_list_final']) > 0 else '',
                'Point 2': trip_order['points_list_final'][1] if len(trip_order['points_list_final']) > 1 else '',
                'Point 3': trip_order['points_list_final'][2] if len(trip_order['points_list_final']) > 2 else '',
                'Point 4': trip_order['points_list_final'][3] if len(trip_order['points_list_final']) > 3 else '',
                'Point 5': trip_order['points_list_final'][4] if len(trip_order['points_list_final']) > 4 else '',
                'Point 6': trip_order['points_list_final'][5] if len(trip_order['points_list_final']) > 5 else '',
                'Scheduled Start (Order)': trip_order['start_datetime_orig'].strftime('%d/%m/%Y %H:%M'),
                'Actual Start Time (Trip)': trip_order['actual_start_time'].strftime('%d/%m/%Y %H:%M') if 'actual_start_time' in trip_order else '',
                'Actual End Time (Trip)': trip_order['actual_end_time'].strftime('%d/%m/%Y %H:%M') if 'actual_end_time' in trip_order else '',
                'NOTES': trip_order['NOTES'],
                'SR': trip_order.get('SR', ''),
                'Inherited Trip Driving (min)': trip_order['inherent_trip_driving_time'],
                'Inherited Ops (min)': trip_order['inherent_operations_time'],
                'Inherited Wait (min)': trip_order['inherent_total_waiting_time'],
                'Inherited Special (min)': trip_order['inherent_special_code_time'],
                'Deadhead To This Trip (min)': deadhead_to_this_trip
            }
            optimized_plan_rows.append(row)

        if driver_block['return_trip_driving_time'] > 0:
            return_start_time = last_order_of_block['actual_end_time']
            return_end_time = return_start_time + timedelta(minutes=driver_block['return_trip_driving_time'])
            row = {
                # 'Driver ID': driver_block['driver_id'], # Removed
                'Block ID': driver_block['block_id'],
                'Trip Order in Block': len(driver_block['orders']) + 1,
                'Trip ID (Order ID)': -1,
                'Type': 'Return to Base',
                'Description': f'Return from {final_location} to {BASE_LOCATION}',
                'Truck Type': 'TR',
                'Point 1': final_location, 'Point 2': BASE_LOCATION,
                'Point 3': '', 'Point 4': '', 'Point 5': '', 'Point 6': '',
                'Scheduled Start (Order)': '',
                'Actual Start Time (Trip)': return_start_time.strftime('%d/%m/%Y %H:%M'),
                'Actual End Time (Trip)': return_end_time.strftime('%d/%m/%Y %H:%M'),
                'NOTES': 'AUTO_GENERATED_RETURN_TRIP', 'SR': '',
                'Inherited Trip Driving (min)': driver_block['return_trip_driving_time'],
                'Inherited Ops (min)': 0, 'Inherited Wait (min)': 0, 'Inherited Special (min)': 0,
                'Deadhead To This Trip (min)': 0
            }
            optimized_plan_rows.append(row)

    optimized_plan_df = pd.DataFrame(optimized_plan_rows)

    # --- Prepare Unassigned Trips Data ---
    unassigned_trips_rows = []
    for order in processed_orders:
        if order['original_order_id'] not in assigned_order_ids:
            unassigned_row = {
                'Order ID': order['original_order_id'],
                'Original DF Index': order['original_df_index'],
                'Type': order['TYPE'],
                'Description': order['DESCRIPTION'],
                'Scheduled Start': order['START_original_str'], # Original string value
                'Earliest Parsed Start': order['earliest_start_dt'].strftime('%d/%m/%Y %H:%M') if pd.notnull(order['earliest_start_dt']) else '',
                'Latest Parsed Start': order['latest_start_dt'].strftime('%d/%m/%Y %H:%M') if pd.notnull(order['latest_start_dt']) else '',
                'Point 1': order['points_list_orig'][0] if len(order['points_list_orig']) > 0 else '',
                'Point 2': order['points_list_orig'][1] if len(order['points_list_orig']) > 1 else '',
                'Point 3': order['points_list_orig'][2] if len(order['points_list_orig']) > 2 else '',
                'Point 4': order['points_list_orig'][3] if len(order['points_list_orig']) > 3 else '',
                'Point 5': order['points_list_orig'][4] if len(order['points_list_orig']) > 4 else '',
                'Point 6': order['points_list_orig'][5] if len(order['points_list_orig']) > 5 else '',
                'NOTES': order['NOTES'],
                'SR': order.get('SR', ''),
                'Reason Unassigned': order.get('reason_unassigned', 'Not processed or no specific reason logged during assignment attempt')
            }
            unassigned_trips_rows.append(unassigned_row)
    unassigned_trips_df = pd.DataFrame(unassigned_trips_rows)

    # --- Prepare Summary Report Data ---
    num_blocks_created = len(drivers_plan) # Each item in drivers_plan is a block
    num_unassigned_trips = len(unassigned_trips_df)

    total_block_duration_minutes = 0
    if num_blocks_created > 0:
        for block in drivers_plan:
            total_block_duration_minutes += block.get('total_working_time_final', 0)
        average_block_duration_minutes = total_block_duration_minutes / num_blocks_created
    else:
        average_block_duration_minutes = 0

    summary_data = {
        'Metric': ['Number of Blocks Created', 'Number of Unassigned Trips', 'Average Block Duration (minutes)'],
        'Value': [num_blocks_created, num_unassigned_trips, f"{average_block_duration_minutes:.2f}"]
    }
    summary_df = pd.DataFrame(summary_data)

    # --- Write to Excel ---
    if not os.path.exists(OUTPUT_FOLDER):
        try:
            os.makedirs(OUTPUT_FOLDER)
            print(f"Created output folder: {OUTPUT_FOLDER}")
        except OSError as e:
            print(f"Error creating output folder {OUTPUT_FOLDER}: {e}")
            # Fallback to current directory if creation fails
            print("Attempting to save output to current directory.")
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            output_file_name = f'delivery_plan_optimized_{timestamp}.xlsx' # Save to script's dir

    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    output_file_name = os.path.join(OUTPUT_FOLDER, f'delivery_plan_optimized_{timestamp}.xlsx')

    try:
        with pd.ExcelWriter(output_file_name, engine='openpyxl') as writer: # openpyxl is common in Colab
            optimized_plan_df.to_excel(writer, sheet_name='Optimized Plan', index=False)
            if not unassigned_trips_df.empty:
                unassigned_trips_df.to_excel(writer, sheet_name='Unassigned Trips', index=False)
            else:
                # Create an empty sheet or a sheet with a message if no unassigned trips
                pd.DataFrame([{'Status': 'No unassigned trips'}]).to_excel(writer, sheet_name='Unassigned Trips', index=False)
            summary_df.to_excel(writer, sheet_name='Summary Report', index=False)
        print(f"Successfully generated multi-sheet delivery plan: {output_file_name}")
    except Exception as e:
        print(f"Error saving Excel file: {e}")
        print("Make sure you have 'openpyxl' installed (pip install openpyxl).")


    # --- Console Evaluation ---
    print(f"\n--- Evaluation ---")
    print(f"Total orders in input file: {len(orders_df_initial)}")
    print(f"Total orders pre-processed (valid for assignment attempt): {len(processed_orders) - skipped_orders_due_to_time_parse}")
    print(f"Total orders assigned to blocks: {len(assigned_order_ids)}")
    print(f"Total orders unassigned: {num_unassigned_trips}")

    if num_unassigned_trips > 0:
        print("First 5 Unassigned order IDs and reasons (if available):")
        for i, row in unassigned_trips_df.head().iterrows():
            print(f"  Order ID: {row['Order ID']}, Reason: {row.get('Reason Unassigned', 'N/A')}")


    print(f"Number of blocks created: {num_blocks_created}")
    print(f"Average block duration: {average_block_duration_minutes:.2f} minutes")

    for i, driver_block in enumerate(drivers_plan): # driver_block is block
        block_start_time_str = ""
        block_end_time_str = ""
        if driver_block['orders']:
            first_order_actual_start = driver_block['orders'][0]['actual_start_time']

            # Calculate initial deadhead for this specific block to find effective start
            initial_deadhead_for_block = 0
            first_order_first_point = driver_block['orders'][0]['points_list_final'][0] if driver_block['orders'][0]['points_list_final'] else None
            if first_order_first_point and first_order_first_point != BASE_LOCATION:
                dist = get_distance(BASE_LOCATION, first_order_first_point)
                if dist != float('inf'):
                    initial_deadhead_for_block = calculate_segment_driving_time(dist)

            effective_block_start = first_order_actual_start - timedelta(minutes=initial_deadhead_for_block)
            block_start_time_str = effective_block_start.strftime('%d/%m/%Y %H:%M')

            last_order_actual_end = driver_block['orders'][-1]['actual_end_time']
            effective_block_end = last_order_actual_end + timedelta(minutes=driver_block.get('return_trip_driving_time',0))
            block_end_time_str = effective_block_end.strftime('%d/%m/%Y %H:%M')


        print(f"  Block ID {driver_block['block_id']}: {len(driver_block['orders'])} trips assigned.")
        if block_start_time_str and block_end_time_str:
             print(f"    Effective Block Time (incl. initial deadhead & final return): {block_start_time_str} to {block_end_time_str}")
        print(f"    Total Driving Time (incl. deadhead & return): {driver_block['total_driving_time_final'] / 60:.2f} hrs")
        print(f"    Total Working Time (calculated for block): {driver_block['total_working_time_final'] / 60:.2f} hrs (Min 8hr target: {driver_block['total_working_time_final'] >= MIN_DAILY_WORKING})")

        used_extended_flag = driver_block.get('used_extended_hours', False) or \
                             driver_block['total_driving_time_final'] > MAX_DAILY_DRIVING_STD or \
                             driver_block['total_working_time_final'] > MAX_DAILY_WORKING_STD

        if driver_block['total_driving_time_final'] > MAX_DAILY_DRIVING_EXT or \
           driver_block['total_working_time_final'] > MAX_DAILY_WORKING_EXT:
            print(f"    WARNING: Block {driver_block['block_id']} EXCEEDED EXTENDED LIMITS! Driving: {driver_block['total_driving_time_final']/60:.2f}h, Working: {driver_block['total_working_time_final']/60:.2f}h")
        elif used_extended_flag:
            print("    Used Extended Hours for this block.")

# This is the crucial part: calling the function to start the process.
if __name__ == "__main__":
    print("Starting delivery optimization process...")
    optimize_deliveries()
    print("Delivery optimization process finished.")


Mounted at /content/drive
Starting delivery optimization process...
Successfully generated multi-sheet delivery plan: /content/drive/My Drive/ASCP PW/2. BLOCKS/Blocks Analysis/delivery_plan_optimized_20250527_082446.xlsx

--- Evaluation ---
Total orders in input file: 48
Total orders pre-processed (valid for assignment attempt): 48
Total orders assigned to blocks: 43
Total orders unassigned: 5
First 5 Unassigned order IDs and reasons (if available):
  Order ID: 24, Reason: Exceeds max continuous driving for new driver (274)
  Order ID: 42, Reason: Exceeds max continuous driving for new driver (274)
  Order ID: 26, Reason: Exceeds max continuous driving for new driver (461)
  Order ID: 43, Reason: Exceeds max continuous driving for new driver (274)
  Order ID: 44, Reason: Exceeds max continuous driving for new driver (461)
Number of blocks created: 28
Average block duration: 505.25 minutes
  Block ID 1: 3 trips assigned.
    Effective Block Time (incl. initial deadhead & final return): 